In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [2]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [4]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [5]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

### Discharge data

In [95]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
fsamp = 5000
def func1_scale(i):
    bb = step_file_objs[i]
        
    freq2 = np.fft.rfftfreq(len(bb.other_data[2]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[2]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
    print(phi_tmp2,np.std(bb.other_data[2])*np.sqrt(2)*100)

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2-3.2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-10 * np.pi, 10 * np.pi], 
                  'limit_A': [-100000000, 100000000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}
    
    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    bb.load_other_data()
    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0], m1_tmp.values[2]-phi_tmp2

In [96]:
func1_scale(0)

[ 2.81969182] 2.40125143269


(-27316.571989536285, array([-3.2]))

In [236]:
step_cal_dir = r'/data/old_trap/20210919/bead1/discharge/check_charge15/'
# step_cal_dir = r'/data/old_trap/20210910/bead1/discharge/fine_4/'
max_file = 500

In [241]:
step_cal_files = []
for root, dirnames, filenames in os.walk(step_cal_dir):
    for filename in fnmatch.filter(filenames, '*' + config.extensions['data']):
        if '_fpga.h5' in filename:
            continue
        step_cal_files.append(os.path.join(root, filename))
step_cal_files.sort(key = bu.find_str)
num_calib_files = len(step_cal_files)
print(num_calib_files)

# Do the step calibration>

step_file_objs = []
for filname in step_cal_files[:]:
    try:
        df = bu.DataFile();
        df.load(filname);
        
        
        df.load_other_data()
        step_file_objs.append(df);
    except:
        print('***')

5


In [242]:
amps4 = []
phases4 = []
for i in range(num_calib_files):
    
    print(i,'/',len(step_cal_files))
    amp, phase = func1_scale(i)
    amps4.append(amp)
    phases4.append(phase)

0 / 5
[ 0.59792832] 2.40117478715
1 / 5
[ 5.53327232] 2.40157225156
2 / 5
[ 2.29429906] 2.39911152341
3 / 5
[ 5.83606359] 2.40359674566
4 / 5
[ 2.58595041] 2.40286064988


In [243]:
_,ax = plt.subplots()
ax.scatter(range(len(phases4)), phases4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [244]:
# amps = np.array(amps)
amps3_tmp = np.array(amps4)
_,ax = plt.subplots(figsize=(9.5,4))
# scale = np.mean(amps4[50:90])/3*0.99

# Perrinf
ax.scatter(range(len(amps3_tmp)), amps3_tmp/scale, label='fit phase')
# ax.scatter(range(len(amps3)), amps3/scale, label='fit fixed-phases')

# ax.set(xlabel='dataset', ylabel='electron charges', ylim=(-0.2,8), xlim=(5,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [130]:
scale

292.49050326857315

In [12]:
scaleZ = scale*-1
print("Scale is ", scaleZ)
bb = step_file_objs[-1]
voltage = np.std(bb.other_data[2])*np.sqrt(2)*100
print("voltage: ", voltage)

Scale is  -292.490503269
voltage:  35.9314358239


In [397]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
def get_amp2(i):
    
    fname = r'/data/old_trap/20210910/bead1/single_electrode/y_electrode6_50Vpp_41Hz_'+str(i)+'.h5'

    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 10000], 'limit_f': [0, 100],
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.other_data[2])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    freq2 = np.fft.rfftfreq(len(bb.other_data[2]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[2]))

    phi_tmp = m1_tmp.values[2]
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
    print(phi_tmp, m1_tmp.values[0]*100)
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp-2.7, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-10000, 10000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [398]:
i=0
get_amp2(i)

Can't find/open HDF5 file : /data/old_trap/20210910/bead1/single_electrode/y_electrode6_50Vpp_41Hz_0.h5


Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45, in getdata
    f = h5py.File(fname,'r')
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 394, in __init__
    swmr=swmr)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 170, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 85, in h5py.h5f.open
OSError: Unable to open file (unable to open file: name = '/data/old_trap/20210910/bead1/single_electrode/y_electrode6_50Vpp_41Hz_0.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45, in getdata
    f = h5py.File(fname,'r')
  File "/home/an

KeyError: 'Time'

In [89]:
amps_elec5 = []
for i in range(20):
    print(i,'/20')
    try:
        amps_elec5.append(get_amp2(i))
    except:
        print("******")
amps_elec5 = np.array(amps_elec5)

0 /20
2.2624597301783584 0.0018430605930319643
1 /20
2.091274835972479 0.001943517291547181
2 /20
-0.18344526200203592 0.0019588708988216297
3 /20
3.3078385541100204 0.0019950590068873453
4 /20
-1.4150070502814085 0.001999924836848521
5 /20
-1.6463525127471739 0.0020120159982539576
6 /20
2.635149412057153 0.0018437735782583786
7 /20
-3.8273011965490866 0.0016930511437251994
8 /20
0.35164270680282605 0.0019967768549733478
9 /20
0.647150232057915 0.0019674503692890255
10 /20
0.17586295841029997 0.0019467317757815294
11 /20
-6.280886274891358 0.020184307503168242
12 /20
0.5376345399919442 0.0019509382442883805
13 /20
2.080224090873317 0.001957290329812622
14 /20
-1.6988046652447517 0.0018456617900675099
15 /20
2.9926664955846984 0.0018210537522378445
16 /20
2.9387131113952734 0.002114460440072463
17 /20
0.1733643797407174 0.0019758457092677872
18 /20
3.074268469721538 0.0017413083752693126
19 /20
-0.25550902468302095 0.0020134809930461017


In [183]:
response_elec3 = np.mean(amps_elec3)
response_elec4 = np.mean(amps_elec4)
print('electrodes response - x: ', response_elec3, response_elec4, response_elec3/response_elec4)

response_elec5 = np.mean(amps_elec5)
# response_elec6 = np.mean(amps_elec6)
# print('electrodes response - y: ', response_elec5, response_elec6, response_elec5/response_elec6)

response_elec1 = np.mean(amps_elec1)
response_elec2 = np.mean(amps_elec2)
print('electrodes response - z: ', response_elec1, response_elec2, response_elec1/response_elec2)

# print('2nd harmonic: ', np.mean(amps_y_82Hz), scaleY)
# response_elec5

electrodes response - x:  3771.36273158 -3328.83483844 -1.13293777391
electrodes response - z:  4931.41581757 -6129.84939733 -0.80449216578


In [174]:
# scaleY = scale*(response_elec4-response_elec5)/(response_elec2-response_elec3)
scaleZ = scale*(response_elec1-response_elec2)/(response_elec3-response_elec4)
print(scale, scaleZ)

-245.761425313 -382.86713548


In [175]:
scale, scaleZ = (-245.761425313, -382.86713548)

In [188]:
_,ax = plt.subplots()
ax.scatter(range(20), np.abs(amps_elec4))
ax.scatter(range(20), np.abs(amps_elec5))
x = np.mean(np.abs(amps_elec4)); dx = np.std(np.abs(amps_elec4))/np.sqrt(20)
y = np.mean(np.abs(amps_elec5)); dy = np.std(np.abs(amps_elec5))/np.sqrt(20)
z = x/y; dz = np.abs(x/y)*np.sqrt((dx/x)**2+(dy/y)**2)
print(x, dx)
print(y, dy)
print(z, dz)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

4278.67333736 9.74884202318
4134.20916265 14.8038215541
1.03494360566 0.00439255803019


### Look at neutrality data

In [504]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
def get_amp(i):
    
    fname = r'/data/old_trap/20210910/bead1/neutrality/z_both_50Vpp_41Hz_delay_10s_spin_xy_25kHz_350Vpp_height_0_files_1000/neutrality_z_50Vpp_41Hz_spin_xy_25kHz_350Vpp_'+str(i)+'.h5'
    
    drive_freq = 41
    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    ## EXCTRACT DIRCE PHASE
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 10000], 'limit_f': [0, 100], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.other_data[2])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    phi_tmp = m1_tmp.values[2]

    ## CHEKC WITH SIMPLE METHOD
#     freq2 = np.fft.rfftfreq(len(bb.other_data[2]), d=1./5000)
#     fft_angles = np.angle(np.fft.rfft(bb.other_data[2]))
#     phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
#     print(phi_tmp, phi_tmp2, m1_tmp.values[0]*100, np.std(xx2)*np.sqrt(2)*100)
    
    ## EXCTRAT FIRST HARMONIC IN-PHASE RESPONSE
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp-2.7, 'sigma': 4000,
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':100,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-100000, 100000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]

    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    ## EXCTRAT SECOND HARMONIC IN-PHASE RESPONSE
    drive_freq = 82
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp+np.pi/2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-np.pi, np.pi], 
                  'limit_A': [0, 100000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

        
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.pos_data[0])[::decimate]
    
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
#     print(m1_tmp.values)
    return m1_tmp.values[0], m1_tmp.values[3], m2_tmp.values[0], m1_tmp.fval

In [505]:
i=1
get_amp(i)

(-3.366012416110607, 4865.498513744532, 6.873981382149452, 17.98110367429522)

In [506]:
i = 0
fname = r'/data/old_trap/20210910/bead1/neutrality/z_both_50Vpp_41Hz_delay_10s_spin_xy_25kHz_350Vpp_height_0_files_1000/neutrality_z_50Vpp_41Hz_spin_xy_25kHz_350Vpp_'+str(i)+'.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
print("voltage: ", voltage, np.mean(bb.other_data[1])*100, np.mean(bb.other_data[2])*100)
drive_freq = 41
bandwidth=10
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[2])
voltage50 = np.std(xx2)*np.sqrt(2)*100
print("voltage: ", voltage50)

voltage:  35.9314358239 -0.577892118715 -0.559357289998
voltage:  23.8474038331


In [507]:
voltage, voltage75, voltage50

(35.931435823903442, 35.771899057134412, 23.847403833072526)

In [515]:
# amps_w_fb_spin_xy_350V_z_50V_delay = []
# for i in range(90):
#     print(i,'/500')
#     try:
#         amps_w_fb_spin_xy_350V_z_50V_delay.append(get_amp(i))
#     except:
#         print("******")
# amps_w_fb_spin_xy_350V_z_50V_delay = np.array(amps_w_fb_spin_xy_350V_z_50V_delay)

In [524]:
amps_w_fb_spin_xy_350V_z_50V_delay = np.array(Parallel(n_jobs=40)(delayed(get_amp)(i) for i in tqdm(range((1000)))))

100%|██████████| 1000/1000 [00:22<00:00, 63.51it/s]


In [453]:
fvals = amps_w_fb_spin_xy_200V_z_75V_delay[:,1]
thres = np.mean(fvals)+2*np.std(fvals)
thres

24578.695627170564

In [478]:
i=0; j=2
_,ax = plt.subplots()
dat = amps_w_fb_spin_xy_200V_z_75V_delay
ax.scatter(dat[:,i], dat[:,j])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [134]:
dat = amps_w_fb_spin_xz_50V_x_50V_delay
thresh = 22
amps_w_fb_spin_xz_50V_x_50V_delay_2 = np.array([aa for aa,bb in zip(dat,dat[:,3]) if (bb<thresh)])

In [135]:
len(amps_w_fb_spin_xz_100V_x_50V_delay_2), len(dat)

(1992, 4000)

In [525]:
j = 0
data = amps_w_fb_spin_xy_200V_z_75V_delay[:,j]/scaleZ*voltage/voltage75
data2 = amps_w_fb_spin_xy_350V_z_50V_delay[:,j]/scaleZ*voltage/voltage50

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data, label='75Vpp-z, xy-spin-200V')
ax.scatter(range(len(data2)), data2, label='75Vpp-z, xy-spin-200V')
ax.plot(np.arange(0,1000), np.arange(0,1000)*0, '--k')

ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print('mean, std: ', np.mean(data), np.std(data)/np.sqrt(len(data)))
print('mean, std: ', np.mean(data2), np.std(data2)/np.sqrt(len(data2)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean, std:  -0.106511722975 0.00320838242352
mean, std:  0.0490217108544 0.00242508557425


In [163]:
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

range_pram = 0.15
_,ax = plt.subplots()
entries, bins, patches = ax.hist(data, bins=30, range=(-range_pram, range_pram), label='50V-x, xz-spin-100V')
entries2, bins, patches = ax.hist(data2, bins=30, range=(-range_pram, range_pram), label='50V-x, xz-spin-50V', alpha=0.6)
entries3, bins, patches = ax.hist(data3, bins=30, range=(-range_pram, range_pram), label='25V-x, xz-spin-100V', alpha=0.6)
ax.set(xlabel=r'$\epsilon$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [168]:
## gaussian fit to the histogram

bins_fit =[(bins[i+1]-bins[i])/2+bins[i] for i in range(len(entries))]

for i,ee in enumerate([entries, entries2, entries3]):
    x = bins_fit
    y= ee/40

    n = len(x)                          #the number of data
    mean = sum(x*y)/n                   #note this correction
    sigma = sum(y*(x-mean)**2)/n        #note this correction

    def gaus(x,a,x0,sigma):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))

    popt,pcov = curve_fit(gaus,x,y,p0=[1,mean,sigma])
    ax.plot(x,gaus(x,*popt)*40,'ro:',label='fit')

    print("mean: ", popt[1], "std: ", np.sqrt(pcov)[1,1])

mean:  -0.000909706807534 std:  0.000522573660436
mean:  -0.0013819508139 std:  0.00057092062916
mean:  -0.000573594379017 std:  0.00132992511715


/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in sqrt
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in sqrt


In [119]:
datasets = [amps_wo_fb_spin_zx_75V_x_20V,amps_wo_fb_spin_zx_100V_x_25V, amps_wo_fb_spin_zx_150V_x_25V]
voltages = [voltage20, voltage25, voltage25]
spin_voltage = [75,100, 150]
means = []
errors = []
for dat,vv in zip(datasets, voltages):
    means.append(np.mean(dat[:,0])/scale*voltage/vv)
    errors.append(np.std(dat[:,0])/scale*voltage/vv/np.sqrt(len(dat[:,0])))

_,ax = plt.subplots()
ax.errorbar(spin_voltage, means, yerr=errors, fmt='o', label='zx plane spin')
ax.set(xlabel='Spin voltage', ylabel=r'$\epsilon$', xlim=(20,160), ylim=(-0.001,0.01))

datasets = [amps_wo_fb_spin_xy_30V_x_20V,amps_wo_fb_spin_xy_75V_x_20V, amps_wo_fb_spin_xy_100V_x_20V]
voltages = [voltage20, voltage20, voltage20]
spin_voltage = [30,75, 100]
means2 = []
errors2 = []
for dat,vv in zip(datasets, voltages):
    means2.append(np.mean(dat[:,0])/scale*voltage/vv)
    errors2.append(np.std(dat[:,0])/scale*voltage/vv/np.sqrt(len(dat[:,0])))

ax.errorbar(spin_voltage, means, yerr=errors, fmt='o', label='xy plane spin')
ax.plot(np.arange(20,160), np.arange(20,160)*0, '--r')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [384]:
j = 0
data = amps_with_fb_spin_xy_100V_x_20V_delay_2[:,j]/scale*voltage/voltage20_1
data2 = amps_wo_fb_spin_zx_100V_x_25V[:,j]/scale*voltage/voltage25

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data, label='20V-x, xy-spin-100V, w/ delay, w/ height fb')
ax.scatter(range(len(data2)), data2, label='25V-x, xy-spin-100V, w/ delay, w/o height fb')

ax.plot(np.arange(0,3000), np.arange(0,3000)*0, '--k')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print('mean, std: ',np.mean(data), np.std(data)/np.sqrt(len(data)))
print('mean, std: ', np.mean(data2), np.std(data2)/np.sqrt(len(data2)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean, std:  0.00543937010689 0.000832138675641
mean, std:  0.00364790172652 0.000627235885947


In [181]:
datasets = [amps_wo_fb_spin_xy_100V_x_20V2[1200:],amps_wo_fb_spin_xy_100V_x_50V_delay2, amps_wo_fb_spin_xy_100V_x_100V_delay2]
spin_voltage = [20,50, 100]
means = []
errors = []
for dat,vv in zip(datasets, voltages):
    means.append(np.mean(dat))
    errors.append(np.std(dat)/np.sqrt(len(dat)))

_,ax = plt.subplots()
ax.errorbar(spin_voltage, means, yerr=errors, fmt='o', label='xy plane spin, 100V')
ax.plot(np.arange(15,110), np.arange(15,110)*0, '--r')
ax.set(xlabel='Oscillating voltage', ylabel=r'$\epsilon$', xlim=(15,110), ylim=(-0.001,0.004))
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [143]:
def get_dcV(i):
    
    fname = r'/data/old_trap/20210715/bead2/neutrality/no_height_fb_electrodes_x_20V_41_Hz_spin_xy_30V_50kHz_files_3000/turbombar_powfb_xyzcool_'+str(i)+'.h5'
    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    return np.mean(bb.other_data[:,5000:85000], axis=1)*100

In [146]:
get_dcV(0)

array([-3.79819842, -0.56254724, -0.57267787,  0.66225783,  0.92968812,
       -0.39438807, -0.0508512 ,  0.22620381])

In [144]:
voltages_elec3_30V_spin = np.array(Parallel(n_jobs=40)(delayed(get_dcV)(i) for i in tqdm(range(3000))))


100%|██████████| 3000/3000 [00:09<00:00, 314.37it/s]

In [313]:
i=1
fname = r'/data/old_trap/20210817/bead1/neutrality/height_fb_0_7_x_41Hz_50V_delay_10s__spin_xz_100V_50kHz_files_3000/x_neutrality_50V_41Hz_'+str(i)+'.h5'
    
drive_freq = 41
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
    
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[3])[::decimate]
m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
xx2 = signal.filtfilt(b, a, bb.pos_data[0])[::decimate]

_,ax = plt.subplots()
ax.scatter(range(50000), bb.other_data[3])

NameError: name 'fit_kwargs' is not defined